In [1]:
import numpy as np
import pandas as pd

### read in the full Amazon file (455,000 rows)

In [2]:
amazon = pd.read_csv('Amazon.csv')
print(amazon.shape)

(455000, 13)


### randomly split the dataset into training and test sets

In [3]:
# create training and test sets
from sklearn.model_selection import train_test_split

raw_data_train, raw_data_test, y_train, y_test = train_test_split(amazon, amazon['helpful'], test_size= 0.2, random_state=7)

### check that the file sizes are what you expected

In [4]:
print(raw_data_train.shape) 
print(raw_data_test.shape)
print(y_train.shape)
print(y_test.shape)

(364000, 13)
(91000, 13)
(364000,)
(91000,)


### export as CSV files so you have readily accessible files to read for training and testing

In [5]:
import os
os.system("mkdir -p /home/eydu/amazon_data/ ") # mw

raw_data_train.to_csv('/home/eydu/amazon_data/raw_data_train.csv')
raw_data_test.to_csv('/home/eydu/amazon_data/raw_data_test.csv')
y_train.to_csv('/home/eydu/amazon_data/y_train.csv')
y_test.to_csv('/home/eydu/amazon_data/y_test.csv')